# We need to map the objects of the image into existing ontology, steps for mapping:

1.finding the suitable ontology for our objects (here, we select the mason ontology)
2.extract the list of ontology classes using owlready2
3.finding the related classes for our objects using similarity measures (sbert similarity: https://www.sbert.net)
4.If there are no matches classes for our object, we will add the class to our ontology using owlready2
5.populate the objects inside of related classes using owlready2
6.create the relations between the objects of one image
7.import the final ontology into Neo4j (I provide you the code)

In [2]:
# !pip install owlready2
# !pip install pyyaml

Load an ontology from a local repository, or from Internet:

In [15]:
#pip install -U sentence-transformers

# 1.finding the suitable ontology for our objects (here, we select the mason ontology)

In [1]:

from owlready2 import *
onto_path.append("")
onto = get_ontology("E://Jacobs 3rd sem//advproj2//main folder//mason.owl")
onto.load()

get_ontology("http://www.owl-ontologies.com/mason.owl#")

In [10]:
from owlready2 import get_ontology, onto_path

onto_path.append("morenap/onto_repo")
onto = get_ontology("mason.owl")
onto.load()

get_ontology("http://www.owl-ontologies.com/mason.owl#")

# 2.extract the list of ontology classes using owlready2

In [2]:

list(onto.classes())

[mason.Manufacturing_operation,
 mason.Finished_part,
 mason.Set,
 mason.System,
 mason.Raw_material,
 mason.Geometric_entity_for_manufacturing,
 mason.Logistic_operation,
 mason.Machine_resource,
 mason.Tool,
 mason.Operation,
 mason.Human_resource,
 mason.Cost_entity,
 mason.Assembly_entity,
 mason.Symmetry,
 mason.Material_resource,
 mason.Geographic_resource,
 mason.Machine_tool,
 mason.Semi-finished_part,
 mason.Topology,
 mason.Geometric_flaw_between_entities,
 mason.Shape,
 mason.Blank,
 mason.Entity_geometric_flaw,
 mason.Resource,
 mason.Geometric_flaw_3rd_order,
 mason.Geometric_flaw_4th_order,
 mason.Administrative_entity,
 mason.Geometric_flaw_2nd_order,
 mason.Technological_entity,
 mason.Part,
 mason.Manufacturing_concept,
 mason.Entity,
 mason.Geometric_entity,
 mason.Geometric_flaw,
 mason.Electropolishing,
 mason.Anodic_dissolving,
 mason.Oxidation,
 mason.Physicochemical_Loss_of_volume,
 mason.Loss_of_volume,
 mason.Addition,
 mason.Mechanical_Small_variations_of_volu

# 3.finding the related classes for our objects using similarity measures (sbert similarity: https://www.sbert.net)

In [3]:
import yaml
from yaml.loader import SafeLoader

In [4]:
with open("E://Jacobs 3rd sem//advproj2//main folder//data.yaml") as f:
    data=yaml.load(f, Loader=SafeLoader)
    print(data)

{'names': ['3DPrinter_Body', '3DPrinter_Filament', 'BandSaw_Controls', 'BandSaw_Saw', 'BandSaw_WorkTable', 'Conveyors_BaseRoller', 'Conveyors_Bed', 'Conveyors_SlantRoller', 'Drillpress_Controls', 'Drillpress_Drill', 'Drillpress_HandWheel', 'Drillpress_MachineStand', 'Drillpress_WorkTable', 'Lathe_Bed', 'Lathe_ChipPan', 'Lathe_Controls', 'Lathe_CrossSlide', 'MillingMachine_ChipCollectorPlate', 'MillingMachine_Controls', 'MillingMachine_HandWheel', 'MillingMachine_MachineStand', 'MillingMachine_WorkTable', 'Robot_BaseArm', 'Robot_Head', 'Robot_UpperArm'], 'nc': 25, 'roboflow': {'license': 'CC BY 4.0', 'project': 'machine_detection', 'url': 'https://universe.roboflow.com/projecttest/machine_detection/dataset/1', 'version': 1, 'workspace': 'projecttest'}, 'path': '../dataset/machine_detection-4', 'test': '../test/', 'train': '../train/', 'val': '../valid/'}


In [5]:
#to get the list of names to extract classes/instances from it
obj_list=data.get('names')
obj_list

['3DPrinter_Body',
 '3DPrinter_Filament',
 'BandSaw_Controls',
 'BandSaw_Saw',
 'BandSaw_WorkTable',
 'Conveyors_BaseRoller',
 'Conveyors_Bed',
 'Conveyors_SlantRoller',
 'Drillpress_Controls',
 'Drillpress_Drill',
 'Drillpress_HandWheel',
 'Drillpress_MachineStand',
 'Drillpress_WorkTable',
 'Lathe_Bed',
 'Lathe_ChipPan',
 'Lathe_Controls',
 'Lathe_CrossSlide',
 'MillingMachine_ChipCollectorPlate',
 'MillingMachine_Controls',
 'MillingMachine_HandWheel',
 'MillingMachine_MachineStand',
 'MillingMachine_WorkTable',
 'Robot_BaseArm',
 'Robot_Head',
 'Robot_UpperArm']

In [6]:
#to get only classes, split each names and get the first word before '_'. Used 'set' to get the distinct vales of classes
cls_lst = set([x.split('_')[0] for x in obj_list])
class_list=[]
#to handle class names starting with a digit. eg(3DPrinter), '_' will be added to it.
for s in cls_lst: class_list.append('_'+s) if s[0].isdigit() else class_list.append(s)
class_list

['MillingMachine',
 'Conveyors',
 'Lathe',
 '_3DPrinter',
 'Robot',
 'BandSaw',
 'Drillpress']

In [2]:
from ontology.utils.ontology import Ontology

temp = Ontology()
onto = temp.set_onto(path="morenap/onto_repo", onto_name="mason.owl")
temp.get_classnames()

[mason.Electropolishing,
 mason.Anodic_dissolving,
 mason.Addition,
 mason.Mechanical_Small_variations_of_volume,
 mason.Water_trimming_machine,
 mason.Shearing_machine-tool,
 mason.Counterbore,
 mason.Revolution_entity,
 mason.Founding_machine,
 mason.Machine-tool,
 mason.Control,
 mason.Manufacturing_operation,
 mason.Tap,
 mason.Forming_machine,
 mason.Punching,
 mason.Cutting,
 mason.Polishing,
 mason.Abrasion,
 mason.Complex_shape_of_revolution_with_draught,
 mason.Complex_shape_of_revolution,
 mason.Electrolytic_ripping,
 mason.Maintenance,
 mason.Logistic_operation,
 mason.Rolling_mill,
 mason.Slot,
 mason.Linear_entity,
 mason.Assembly,
 mason.Machining_centre_5_axis,
 mason.Circular_slot,
 mason.Oxidation,
 mason.Physicochemical_Loss_of_volume,
 mason.Operation,
 mason.Manufacturing_concept,
 mason.Forging,
 mason.Brutal_hot_forming,
 mason.Forging_die_and_punch,
 mason.Tool,
 mason.Administrative_entity,
 mason.Entity,
 mason.Set,
 mason.Assembly_entity,
 mason.Cost_entity,
 

In [6]:
#Create Image class if not present already- onetime thing

class Images(onto.Machine_resource):
    namespace = onto

In [5]:
# !pip install sentence_transformers

In [1]:
from ontology.utils.ontology import Ontology, Labels

labels_list = Labels().load_label_from_yaml("detection/dataset/data.yaml")
labels_list

mason = Ontology("morenap/onto_repo", "mason.owl")
mason.get_onto
mason.semetic_search(labels_list, save_path="results/detection")


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


{'Robot': {'mason.Machine-tool': 0.3474141061306,
  'mason.Folding_machine': 0.3307335674762726,
  'mason.Grinding_machine': 0.3095781207084656},
 'Printer3D': {'mason.Plasma_trimming_machine': 0.27975884079933167,
  'mason.Laser': 0.25432607531547546,
  'mason.Machining_centre_5_axis': 0.24805979430675507},
 'Conveyors': {'mason.Hydroforming': 0.32675573229789734,
  'mason.Manufacturing_concept': 0.3115306794643402,
  'mason.Manual_moulding': 0.3105659484863281},
 'Lathe': {'mason.Lathe': 0.7013975381851196,
  'mason.Extrusion': 0.4131752848625183,
  'mason.Alloy': 0.4047287106513977},
 'Drillpress': {'mason.Drilling': 0.5551064014434814,
  'mason.Drill': 0.5074068307876587,
  'mason.Machine-tool': 0.46587568521499634},
 'MillingMachine': {'mason.Milling_machine': 0.7764980792999268,
  'mason.Milling_machine_4_axis': 0.7418296933174133,
  'mason.Milling': 0.7114167809486389},
 'BandSaw': {'mason.Board_drop_hammer': 0.29878121614456177,
  'mason.Welding_aluminium': 0.29369494318962097,

In [4]:
mason.get_classnames()[]

'mason.Milling_machine'

In [19]:
from ontology.utils.ontology import Ontology


from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
# abc = list(onto.classes())
corpus = [str(a_class) for a_class in onto.classes()]
# corpus = abc
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = OntoLabels().load_from_yaml("detection/dataset/data.yaml")  # Class list
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(3, len(corpus))
for query in queries:
    print(query)
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """

Drillpress




Query: Drillpress

Top 5 most similar sentences in corpus:
mason.Drilling (Score: 0.5551)
mason.Drill (Score: 0.5074)
mason.Machine-tool (Score: 0.4659)
Lathe




Query: Lathe

Top 5 most similar sentences in corpus:
mason.Lathe (Score: 0.7014)
mason.Extrusion (Score: 0.4132)
mason.Alloy (Score: 0.4047)
BandSaw




Query: BandSaw

Top 5 most similar sentences in corpus:
mason.Board_drop_hammer (Score: 0.2988)
mason.Welding_aluminium (Score: 0.2937)
mason.Shearing_machine-tool (Score: 0.2744)
MillingMachine




Query: MillingMachine

Top 5 most similar sentences in corpus:
mason.Milling_machine (Score: 0.7765)
mason.Milling_machine_4_axis (Score: 0.7418)
mason.Milling (Score: 0.7114)
Printer3D




Query: Printer3D

Top 5 most similar sentences in corpus:
mason.Plasma_trimming_machine (Score: 0.2798)
mason.Laser (Score: 0.2543)
mason.Machining_centre_5_axis (Score: 0.2481)
Conveyors




Query: Conveyors

Top 5 most similar sentences in corpus:
mason.Hydroforming (Score: 0.

# 4.If there are no matches classes for our object, we will add the class to our ontology using owlready2

In [27]:
#not manually- similarity measure(sementic+string) 
#as of now super class is thing threshold is 0.5....

#from owlready2 import *
#onto = get_ontology("http://test.org/onto.owl")  


# 5.populate the objects inside of related classes using owlready2

In [69]:

#not manually- list of classes/objects using for loop.

# BandSaw_Controls = Band_Saw("BandSaw_Controls")


In [23]:
# destroy_entity(onto.Lathe_Ch_Controls)

In [3]:
#Access the classes of the ontology, and create new instances / individuals:
onto.Machine_Tool

In [6]:
print(onto.Machine_tool)

mason.Machine_tool


In [8]:
list(onto.object_properties())

[mason.previousOperation,
 mason.assembles,
 mason.isAPartOfAssembly,
 mason.isMachinableByProcess,
 mason.allowedProcessFor,
 mason.enablesRealisationOf,
 mason.requiresMachine,
 mason.toolUsableOn,
 mason.isMachinableWithTool,
 mason.requiredToolFor,
 mason.requiresTool,
 mason.isExecutedBy,
 mason.execute,
 mason.isInducedBy,
 mason.induces,
 mason.hasSymmetry,
 mason.contains,
 mason.isLocatedIn,
 mason.includes,
 mason.isIn,
 mason.usesTool,
 mason.isUsedBy,
 mason.becomes,
 mason.ofTopology,
 mason.hasShape,
 mason.isMadeOf]

In [11]:
list(onto.data_properties())

[mason.identifier,
 mason.rugosity,
 mason.duration,
 mason.batch_run_size,
 mason.corrugation,
 mason.name,
 mason.complexity_rating,
 mason.delay,
 mason.dimension_X,
 mason.dimension_Y,
 mason.operating_rate,
 mason.dimension_Z,
 mason.volume,
 mason.filling_rate]

In [10]:
list(onto.annotation_properties())

[]

In [55]:
list(onto.properties())

[mason.identifier,
 mason.rugosity,
 mason.duration,
 mason.batch_run_size,
 mason.corrugation,
 mason.name,
 mason.complexity_rating,
 mason.delay,
 mason.dimension_X,
 mason.dimension_Y,
 mason.operating_rate,
 mason.dimension_Z,
 mason.volume,
 mason.filling_rate,
 mason.previousOperation,
 mason.assembles,
 mason.isMachinableByProcess,
 mason.enablesRealisationOf,
 mason.toolUsableOn,
 mason.isMachinableWithTool,
 mason.requiredToolFor,
 mason.requiresTool,
 mason.isExecutedBy,
 mason.execute,
 mason.isInducedBy,
 mason.induces,
 mason.hasSymmetry,
 mason.contains,
 mason.isLocatedIn,
 mason.includes,
 mason.isIn,
 mason.isAPartOfAssembly,
 mason.usesTool,
 mason.isUsedBy,
 mason.allowedProcessFor,
 mason.becomes,
 mason.requiresMachine,
 mason.ofTopology,
 mason.hasShape,
 mason.isMadeOf]

In [16]:
list(onto.disjoint_classes() )

[]

In [18]:
list(onto.disjoint_properties())

[]

In [19]:
list(onto .disjoints() )

[]

In [11]:
list(onto.different_individuals())

[]

In [28]:
#delete class
# destroy_entity(onto.Images)

In [44]:
 #to get parent class
print(onto.Images.is_a)

[mason.Machine_resource]


In [43]:
# Owlready classes and individuals can be used as values within search(), as follows:
onto.search_one(is_a=onto.Images)

mason.Images

In [46]:
#to check the flow of parent and subclasses # .descendants()#.ancestors()
# Robots.descendants() 

In [7]:
list(onto.individuals())

[]

In [53]:
##save ontology
onto.save()
try:
    onto.save(file = "./ontology_mason.rdf", format = "rdfxml")
except ValueError:
    print(ValueError)
# onto.save(file = "test_mason", format = "rdfxml")

In [ ]:
#To put mason ontology on neo4j#neo4jconnection #used to be COnnect2Neo4j

# 6.create the classes from images dynamically


In [ ]:
# first-extract json files data

In [8]:
import json
import glob
from datetime import datetime
# Place your JSON data in a directory named 'data/'
src = "data/"

date = datetime.now()
data = []

# Change the glob if you want to only look through files with specific names
files = glob.glob('data/*', recursive=True)
print('file is:',files)
# Loop through files

for single_file in files:
    
#     print(single_file)
    with open(single_file, 'r') as f:

        # Use 'try-except' to skip files that may be missing data
        try:
            
            
            json_file = json.load(f)
            data.append([
              json_file['img_id'],
              json_file['img_name'],
#               json_file['detection'],
#               json_file['detection'][0][1],
              [item[1] for item in json_file['detection']]  
            ])
        except KeyError:
            print(f'Skipping {single_file}')

# Sort the data
data.sort()

# Add headers
# data.insert(0, ['img_id','img_name','detection'])
data



file is: ['data\\IMG_0197.json', 'data\\IMG_0198.json', 'data\\IMG_0204.json']


[['7aaeffd59beadd670d3f7d150420a9e34cc515887ae7c9a224cc2d30d626947c',
  'IMG_0197',
  ['Robot_UpperArm',
   'Robot_Head',
   'Robot_UpperArm',
   'Robot_UpperArm',
   'Robot_BaseArm']],
 ['9f8899036628c22774625074265061f45cc128a703b155f585366f8597c55ee1',
  'IMG_0198',
  ['Robot_UpperArm',
   'Robot_Head',
   'Robot_UpperArm',
   'Robot_BaseArm',
   'Robot_UpperArm']],
 ['a366c7172d098c9c00576e46d648fc46ef0fcbb3b250e286e4b2262ec99db186',
  'IMG_0204',
  ['3DPrinter_Filament',
   '3DPrinter_Body',
   'Robot_Head',
   '3DPrinter_Filament',
   'Robot_UpperArm',
   '3DPrinter_Filament',
   'Robot_BaseArm',
   'Robot_UpperArm',
   'Robot_Head']]]

In [9]:
imgClasses= ['_'+item[0] for item in data]  
imgObjects= [item[2] for item in data]   #[set(item[2]) for item in data]  
print(imgClasses,' ',imgObjects)

['_7aaeffd59beadd670d3f7d150420a9e34cc515887ae7c9a224cc2d30d626947c', '_9f8899036628c22774625074265061f45cc128a703b155f585366f8597c55ee1', '_a366c7172d098c9c00576e46d648fc46ef0fcbb3b250e286e4b2262ec99db186']   [['Robot_UpperArm', 'Robot_Head', 'Robot_UpperArm', 'Robot_UpperArm', 'Robot_BaseArm'], ['Robot_UpperArm', 'Robot_Head', 'Robot_UpperArm', 'Robot_BaseArm', 'Robot_UpperArm'], ['3DPrinter_Filament', '3DPrinter_Body', 'Robot_Head', '3DPrinter_Filament', 'Robot_UpperArm', '3DPrinter_Filament', 'Robot_BaseArm', 'Robot_UpperArm', 'Robot_Head']]


In [10]:
# destroy_entity(onto._9f8899036628c22774625074265061f45cc128a703b155f585366f8597c55ee1)
# destroy_entity(onto._7aaeffd59beadd670d3f7d150420a9e34cc515887ae7c9a224cc2d30d626947c)
# destroy_entity(onto.Robot_UpperArm_0)
# destroy_entity(onto.Robot_Head_1)
# destroy_entity(onto.Robot_UpperArm_2)
# destroy_entity(onto.Robot_UpperArm_3)
# destroy_entity(onto.Robot_BaseArm_4)
# destroy_entity(onto.Robot_BaseArm_3)
# destroy_entity(onto.Robot_UpperArm_4)
# destroy_entity(onto.3DPrinter_Filament_0)
# destroy_entity(onto.3DPrinter_Body_1)
# destroy_entity(onto.Robot_UpperArm_4)

In [78]:
# destroy_entity(onto.Robot_UpperArm)
# destroy_entity(onto.Robot_Head)
# destroy_entity(onto.Robot_BaseArm)

In [64]:
# def createNewClass(createClass):
#     # To do: Create New Class
#     y = createClass
#     SuperClass =  onto.Images #static value (root class)
#     for index, val in enumerate(createClass):
#         with onto:
# #             print(val, index)
#             yvalue = y[index]
#             NewClass = types.new_class(yvalue, (onto.Images,))
            
# createNewClass(imgClasses)

In [11]:
def createNewClass(createClass,createobject):
    # To do: Create New Class
    y = createClass
    z = createobject
    SuperClass =  onto.Images #static value (root class)
    for index, val in enumerate(createClass):
        print(val, index)
#     for index, val in enumerate(createobject):
        with onto:
            yvalue = y[index]
            zvalue = z[index]
            NewClass = types.new_class(yvalue, (onto.Images,))
            for index, val in enumerate(zvalue):
                print('_'+str(index)+'_'+val)
                my_obj = NewClass('_'+str(index)+'_'+val)
    
createNewClass(imgClasses,imgObjects)

_7aaeffd59beadd670d3f7d150420a9e34cc515887ae7c9a224cc2d30d626947c 0
_0_Robot_UpperArm
_1_Robot_Head
_2_Robot_UpperArm
_3_Robot_UpperArm
_4_Robot_BaseArm
_9f8899036628c22774625074265061f45cc128a703b155f585366f8597c55ee1 1
_0_Robot_UpperArm
_1_Robot_Head
_2_Robot_UpperArm
_3_Robot_BaseArm
_4_Robot_UpperArm
_a366c7172d098c9c00576e46d648fc46ef0fcbb3b250e286e4b2262ec99db186 2
_0_3DPrinter_Filament
_1_3DPrinter_Body
_2_Robot_Head
_3_3DPrinter_Filament
_4_Robot_UpperArm
_5_3DPrinter_Filament
_6_Robot_BaseArm
_7_Robot_UpperArm
_8_Robot_Head


In [12]:
list(onto.individuals())

[mason._0_Robot_UpperArm,
 mason._1_Robot_Head,
 mason._2_Robot_UpperArm,
 mason._3_Robot_UpperArm,
 mason._4_Robot_BaseArm,
 mason._3_Robot_BaseArm,
 mason._4_Robot_UpperArm,
 mason._0_3DPrinter_Filament,
 mason._1_3DPrinter_Body,
 mason._2_Robot_Head,
 mason._3_3DPrinter_Filament,
 mason._5_3DPrinter_Filament,
 mason._6_Robot_BaseArm,
 mason._7_Robot_UpperArm,
 mason._8_Robot_Head]

In [13]:
 for i in onto._9f8899036628c22774625074265061f45cc128a703b155f585366f8597c55ee1.instances(): print(i) #for image 0198

mason._0_Robot_UpperArm
mason._1_Robot_Head
mason._2_Robot_UpperArm
mason._3_Robot_BaseArm
mason._4_Robot_UpperArm


In [14]:
 for i in onto._7aaeffd59beadd670d3f7d150420a9e34cc515887ae7c9a224cc2d30d626947c.instances(): print(i) #for image 0197

mason._0_Robot_UpperArm
mason._1_Robot_Head
mason._2_Robot_UpperArm
mason._3_Robot_UpperArm
mason._4_Robot_BaseArm


In [15]:
 for i in onto._a366c7172d098c9c00576e46d648fc46ef0fcbb3b250e286e4b2262ec99db186 .instances(): print(i) #for image 0204

mason._4_Robot_UpperArm
mason._0_3DPrinter_Filament
mason._1_3DPrinter_Body
mason._2_Robot_Head
mason._3_3DPrinter_Filament
mason._5_3DPrinter_Filament
mason._6_Robot_BaseArm
mason._7_Robot_UpperArm
mason._8_Robot_Head


In [16]:
# onto.Robot_Head.class()
onto._5_3DPrinter_Filament.is_a

[mason._a366c7172d098c9c00576e46d648fc46ef0fcbb3b250e286e4b2262ec99db186]

# relations between the objects of one image

# 7.import the final ontology into Neo4j (I provide you the code)

In [106]:
class Neo4j_connection: #neo4jconnection #used to be COnnect2Neo4j

    def __init__(self, uri, user, pwd, database_name="neo4j"): #connect to the server; create constraint if not exist; default database is "Neo4j"
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        self.__database_name = database_name

        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create driver:", e)

         # check if the neccessary config already existed. if not, create.
        query_string = '''
        CALL db.constraints()
                 '''
        a = self.query(query_string, db= self.__database_name)
        constr = str(a)
        try:
            b = constr[constr.find("description='")+13:constr.find(' ON')]
        except Exception as e:
            pass
        if b == 'CONSTRAINT':
            print("Constraint already existed")
        else:
            try:
                query_string = '''
                    CREATE CONSTRAINT n10s_unique_uri ON (r:Resource)
                    ASSERT r.uri IS UNIQUE  
                            '''
                self.query(query_string, db = self.__database_name)
            except Exception as e:
                print("Failed to create constraint - check again", e)

        # check if the neccessary config already existed. if not, create.
        query_string = '''
        match(n) return count(n)
        '''
        a = self.query(query_string, db= self.__database_name)
        strin = str(a[0])
        config_check = int(strin[strin.find("=")+1:strin.find(">")])
        if config_check != 0:
            print("Config already existed")
        else:
            try: #setting up neccessary config for neosemantics
                query_string = '''
                    CALL n10s.graphconfig.init()
                                '''
                self.query(query_string, db= self.__database_name)

            except Exception as e:
                print("Failed to initiate graphconfig: ", e)


    def uploading_orx(self, address, db=None): #uploading OWL, RDF, and XML #i am trying to
        query_string = (
            "call n10s.rdf.import.fetch(" + f'{address}'+  ',"RDF/XML")')
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query_string))
            print("Sucessfully uploaded XML/RDF/OWL file ")
        except Exception as e:
            print("Upload failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


    def uploading_ttl(self, address, db=None): #uploading Turtle files
        query_string = (
            "call n10s.rdf.import.fetch(" + f'{address}'+  ',"Turtle")')
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query_string))
            print("Sucessfully uploaded TTL file ")
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, db=None): #query commands of choice.
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

if __name__ == "__main__": 
    greeter = Neo4j_connection("bolt://localhost:7687", "mason", "password")
    greeter.uploading_orx("'file:///Users/ABHISHEK MATHUR/ontology_mason'", "mason")
    greeter.close()



Failed to create driver: name 'GraphDatabase' is not defined


AssertionError: Driver not initialized!

In [103]:
#Upload ontology to Neo4j for graphing 
class Neo4j_connection: #neo4jconnection #used to be COnnect2Neo4j

    def __init__(self, uri, user, pwd, database_name="neo4j"): #connect to the server; create constraint if not exist; default database is "Neo4j"
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        self.__database_name = database_name

        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create driver:", e)

        # check if the neccessary config already existed. if not, create.
        query_string = '''
        CALL db.constraints()
                 '''
        a = self.query(query_string, db= self.__database_name)
        constr = str(a)
        try:
            b = constr[constr.find("description='")+13:constr.find(' ON')]
        except Exception as e:
            pass
        if b == 'CONSTRAINT':
            print("Constraint already existed")
        else:
            try:
                query_string = '''
                    CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE  
                            '''
                self.query(query_string, db = self.__database_name)
            except Exception as e:
                print("Failed to create constraint - check again", e)

        # check if the neccessary config already existed. if not, create.
        query_string = '''
        match(n) return count(n)
        '''
        a = self.query(query_string, db= self.__database_name)
        strin = str(a[0])
        config_check = int(strin[strin.find("=")+1:strin.find(">")])
        if config_check != 0:
            print("Config already existed")
        else:
            try: #setting up neccessary config for neosemantics
                query_string = '''
                    CALL n10s.graphconfig.init()
                                '''
                self.query(query_string, db= self.__database_name)

            except Exception as e:
                print("Failed to initiate graphconfig: ", e)


    def uploading_orx(self, address, db="neo4j"): #uploading OWL, RDF, and XML #i am trying to
        query_string = (
                "call n10s.rdf.import.fetch('"+address+"','RDF/XML',"+"{"+"verifyUriSyntax: false"+"}"+")")
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query_string))
            print("Sucessfully uploaded XML/RDF/OWL file ")
        except Exception as e:
            print("Upload failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


    def uploading_ttl(self, address, db=None): #uploading Turtle files
        query_string = (
                "call n10s.rdf.import.fetch(" + f'{address}'+  ',"Turtle",{verifyUriSyntax: false})')
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query_string))
            print("Sucessfully uploaded TTL file ")
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, db=None): #query commands of choice.
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

a = Neo4j_connection("bolt://localhost:7687", "mason", "password") #("bolt://localhost:7687", "neo4j", "1234567")
a.uploading_orx("file:///Users/ABHISHEK MATHUR/ontology_mason.rdf", "mason")
#("file:///Users/jhealynsamson/AP1/ontology_template.rdf","neo4j")

Failed to create driver: name 'GraphDatabase' is not defined


AssertionError: Driver not initialized!